# updated content since there are new libraries to replace the old ones

overview:
 - FastAPI for server
 - uv for package management
 - docker for contanerization
 - fly.io for deployment
 - sklearn pipelines to make it more managable

 ---

## 1. Save and load the model

- assume model is saved already
- import pickle so you can save the model in a file.

In [ ]:
import pickle

with open('model.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [ ]:
# to load


with open('model.bin', 'rb') as f_in:
    (dv, model) = pickle.load(f_in)

## 2. sklearn pipelines.

- It is not conveneient to deal with both dv and modle, so lets ocmbine them into one.
- we can do this instead:


instead of this

```

dv = DictVectorizer()

train_dict = df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
```

you can do this

```

from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    DictVectorizer(),
    LogisticRegression(solver='liblinear')
)

pipeline.fit(train_dict, y_train)
```

and to predict:

`pipeline.predict_proba(datapoint)[0, 1]`

In [ ]:
# and now with pickle

import pickle

with open('model.bin', 'wb') as f_out:
    pickle.dump(pipeline, f_out)

In [ ]:
# to load


with open('model.bin', 'rb') as f_in:
    pipeline = pickle.load(f_in)

## 3. load this whole thing in a python script instead a notebook

then we will need to seperate it into a train and a predict file. here are things to take note of:
- make the load data, train model, and save model in their own functions, call them, and then log something like "train complete"






## 4. Make a FastAPI app

1. is to install fastapi and uvicorn
2. Lets make a simple we app first, and call it ping.py as a proof of concept

In [ ]:
# here is the boilerplate

from fastapi import FastAPI
import uvicorn

app = FastAPI(title="ping")

@app.get("/ping")
def ping():
    return "PONG"

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=9696)

to run the app, you can just do `python ping.py`

but here is the proper way:

`uvicorn ping:app --host 0.0.0.0 --port 9696 --reload`

you can curl it like this:
`curl localhost:9696/ping`

note that in /docs route in fastapi you can see its docs

## 5. turning the script into a web app

In [ ]:
import pickle
from fastapi import FastAPI
import uvicorn

app = FastAPI(title="customer-churn-prediction")

with open('model.bin', 'rb') as f_in:
    pipeline = pickle.load(f_in)


def predict_single(customer):
    result = pipeline.predict_proba(customer)[0, 1]
    return float(result)


@app.post("/predict")
def predict(customer):
    prob = predict_single(customer)

    return {
        "churn_probability": prob,
        "churn": bool(prob >= 0.5)
    }


if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=9696)

In [ ]:
# we can add type hinting

from typing import Dict, Any

@app.post("/predict")
def predict(customer: Dict[str, Any]):
    prob = predict_single(customer)

    return {
        "churn_probability": prob,
        "churn": bool(prob >= 0.5)
    }

## 6. Make a script to test this



In [1]:
import requests

url = 'http://localhost:9696/predict'

customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

response = requests.post(url, json=customer)
predictions = response.json()

print(predictions)
if predictions['churn']:
    print('customer is likely to churn, send promo')
else:
    print('customer is not likely to churn')

{'churn_probability': 0.6638108546481684, 'churn': True}
customer is likely to churn, send promo


## 7. Pydantic and validation

fastapi comes with pydantic. but we can add this bit of code in our app to be more explicit with the input and output type

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

class Customer(BaseModel):
    gender: Literal["male", "female"]
    seniorcitizen: Literal[0, 1]
    partner: Literal["yes", "no"]
    dependents: Literal["yes", "no"]
    phoneservice: Literal["yes", "no"]
    multiplelines: Literal["no", "yes", "no_phone_service"]
    internetservice: Literal["dsl", "fiber_optic", "no"]
    onlinesecurity: Literal["no", "yes", "no_internet_service"]
    onlinebackup: Literal["no", "yes", "no_internet_service"]
    deviceprotection: Literal["no", "yes", "no_internet_service"]
    techsupport: Literal["no", "yes", "no_internet_service"]
    streamingtv: Literal["no", "yes", "no_internet_service"]
    streamingmovies: Literal["no", "yes", "no_internet_service"]
    contract: Literal["month-to-month", "one_year", "two_year"]
    paperlessbilling: Literal["yes", "no"]
    paymentmethod: Literal[
        "electronic_check",
        "mailed_check",
        "bank_transfer_(automatic)",
        "credit_card_(automatic)",
    ]
    tenure: int = Field(..., ge=0)
    monthlycharges: float = Field(..., ge=0.0)
    totalcharges: float = Field(..., ge=0.0)


class PredictResponse(BaseModel):
    churn_probability: float
    churn: bool

In [ ]:
# then we can do this so that we get a specific type for the input and output
@app.post("/predict")
def predict(customer: Customer) -> PredictResponse:
    prob = predict_single(customer.model_dump())

    return PredictResponse(
        churn_probability=prob,
        churn=prob >= 0.5
    )

In [ ]:
# to make it so that it raises an error when there is an issue

from pydantic import ConfigDict


class Customer(BaseModel):
    model_config = ConfigDict(extra="forbid")

    ... # rest of the fields

## 8. Environment management with uv

uv is a tool for dependency and environment management built with rust. This makes it way faster than anything else

to install: `pip install uv`
to initialize: `uv init`

after init, we will realize that we don't need main.py, so we can remove it with rm main.py

now we will have a pyproject.toml file. This file can be used to define our dependencies and such. 

to add dependencies, use` uv add dependency`

there will be a new file called uv.lock that has all the dependency trees

you can use `uv add --dev dependencyName` to add a dev dependency, and `uv run myNormalRunCommand` like 
`uv run uvicorn predict:app --host 0.0.0.0 --port 9696 --reload` to run something with the uv environment

do `uv sync` when theres a new project. this is simlar to npm install. --locked makes sure the installed stuff is exactly what sin uv.lock


## 9. contanerization

sometimes there are system dependencies which are outside of venv that we also need to isolate. This is where we use docker for.

having a container makes it so that it is self contained and runs everywhere



In [ ]:
FROM python:3.12.1


# we always want to start with the python version then install the package manager.
RUN pip install uv

# then we set the working environment that is inside the container
WORKDIR /app


# then we copy python version, pyproject, and uvlock into our ./(working directory)
COPY .python-version pyproject.toml uv.lock ./

# install dependencies
RUN uv sync --locked


# copy our web service and our model into our working directory
COPY main.py model.bin ./

# let docker know that we will run something on this port
EXPOSE 9696


# sets the command to run the web service
ENTRYPOINT [ "uv", "run", "uvicorn", "main:app", "--host", "0.0.0.0","--port", "9696"]

Then we need to build and run this

`docker build -t predict-churn .`  predict-churn is the tag given to this build, . is the current working directory

then we run it 
`docker run -it --rm -p 9696:9696 predict-churn`
- it stands for interactive
- rm means we remove the container when we are done
- -p 9696 makes the port 9696 on the computer the port 9696 in the environment
- predict-churn is the name of the tag

10. Deploy on fly.io

im not adding a credit card but just follow the steps 